In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
import numpy as np



/Users/amit/anaconda2/envs/myenv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/amit/anaconda2/envs/myenv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/amit/anaconda2/envs/myenv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/amit/anaconda2/envs/myenv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [18]:


tf.reset_default_graph()
with tf.Session() as sess:
    x=tf.Variable(1.)
    x=x+x
    y=x
    x=2*x
    sess.run(tf.global_variables_initializer())
    VS=tf.trainable_variables()
    for v in VS:
        print(v,v.eval())
    print(x.eval())
    print(y.eval())

<tf.Variable 'Variable:0' shape=() dtype=float32_ref> 1.0
4.0
2.0


In [9]:
tfe = tf.contrib.eager
w = tfe.Variable([[1.0]])
with tf.GradientTape() as tape:
  loss = w * w

print(loss)
w.assign([[4.]])
print(w)
grad = tape.gradient(loss, w)
print(grad)  # => tf.Tensor([[ 2.]], shape=(1, 1), dtype=float32)


tf.Tensor([[1.]], shape=(1, 1), dtype=float32)
<tf.Variable 'Variable:0' shape=(1, 1) dtype=float32, numpy=array([[4.]], dtype=float32)>
tf.Tensor([[2.]], shape=(1, 1), dtype=float32)


In [2]:
tfe = tf.contrib.eager
model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, input_shape=(784,)),  # must declare input shape
  tf.keras.layers.Dense(10)
])


/Users/amit/anaconda2/envs/myenv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
print(model)

In [5]:
@tf.custom_gradient
def log1pexp(x):
  e = tf.exp(x)
  def grad(dy):
    return dy * (1 - 1 / (1 + e))
  return tf.log(1 + e), grad

grad_log1pexp = tfe.gradients_function(log1pexp)

# As before, the gradient computation works fine at x = 0.
print(grad_log1pexp(0.))  # => [0.5]

# And the gradient computation also works at x = 100.
print(grad_log1pexp(100.))  # => [1.0]


[<tf.Tensor: id=78, shape=(), dtype=float32, numpy=0.5>]
[<tf.Tensor: id=88, shape=(), dtype=float32, numpy=1.0>]


In [6]:
def log1pexp(x):
  return tf.log(1 + tf.exp(x))
grad_log1pexp = tfe.gradients_function(log1pexp)

# The gradient computation works fine at x = 0.
print(grad_log1pexp(0.))  # => [0.5]

# However, x = 100 fails because of numerical instability.
print(grad_log1pexp(100.))  # => [nan]


[<tf.Tensor: id=96, shape=(), dtype=float32, numpy=0.5>]
[<tf.Tensor: id=104, shape=(), dtype=float32, numpy=nan>]


In [9]:
import tensorflow as tf
from tensorflow.python.framework import ops
import numpy as np

# Define custom py_func which takes also a grad op as argument:
def py_func(func, inp, Tout, stateful=True, name=None, grad=None):
    
    # Need to generate a unique name to avoid duplicates:
    rnd_name = 'PyFuncGrad' + str(np.random.randint(0, 1E+8))
    
    tf.RegisterGradient(rnd_name)(grad)  # see _MySquareGrad for grad example
    g = tf.get_default_graph()
    with g.gradient_override_map({"PyFunc": rnd_name}):
        return tf.py_func(func, inp, Tout, stateful=stateful, name=name)
    
def ddd(x,y):
    return(np.dot(x,y))
    
# Def custom square function using np.square instead of tf.square:
def mydot(x, name=None):
    
    with ops.name_scope(name, "Mysquare", [x]) as name:
        dot_xy = py_func(ddd,
                        [x,y],
                        [tf.float32],
                        name=name,
                        grad=_MyDotGrad)  # <-- here's the call to the gradient
        return sqr_x[0]

# Actual gradient:
def _MyDotGrad(op, grad):
    x = op.inputs[0]
    return grad * 10 * x  # add a "small" error just to see the difference:

with tf.Session() as sess:
    x = tf.constant([1., 2.])
    y = mysquare(x)
    tf.global_variables_initializer().run()

    print(x.eval(), y.eval(), tf.gradients(y, x)[0].eval())

[1. 2.] [1. 4.] [10. 20.]


In [53]:
import tensorflow as tf
from tensorflow.python.framework import ops
import numpy as np

tf.reset_default_graph()
a=tf.Variable(np.random.rand(5,6),"a")
b=tf.Variable(np.random.rand(6,1),"b")
c=tf.Variable(np.random.rand(5,6),"c")
#b=tf.Variable(6.,"a")

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('a',a.eval())
    print('c',c.eval())
    print('b',b.eval())
    lossab=tf.reduce_sum(tf.matmul(a,b))
    losscb=tf.reduce_sum(tf.matmul(c,b))
    print('lossab',lossab.eval())
    print('losscb',losscb.eval())

    gradab=tf.gradients(lossab,[b])
    gradcb=tf.gradients(losscb,[b])
    
    print(gradab[0].eval())
    print(gradcb[0].eval())

a [[0.90305383 0.79923935 0.43836708 0.02828587 0.79660186 0.74410462]
 [0.53591689 0.70329835 0.87789272 0.06635842 0.71036043 0.96535111]
 [0.02301801 0.8350919  0.71760156 0.3623323  0.35450563 0.8573271 ]
 [0.04401364 0.53328624 0.43455272 0.53601437 0.43941282 0.7134037 ]
 [0.59063146 0.15909857 0.87981775 0.23979562 0.48883085 0.76140063]]
c [[0.97263005 0.71296995 0.89880469 0.69892602 0.89331679 0.55664304]
 [0.85425185 0.68204215 0.02848011 0.70656992 0.67456518 0.3857768 ]
 [0.46465487 0.68939404 0.69556705 0.33168079 0.52856626 0.40907122]
 [0.68313617 0.7218877  0.0489139  0.30204473 0.00480236 0.87539432]
 [0.70129178 0.30340741 0.87525314 0.15383944 0.91328548 0.31331791]]
b [[0.77112767]
 [0.86710884]
 [0.6446378 ]
 [0.50476649]
 [0.28629879]
 [0.24999904]]
lossab 8.833874799734375
losscb 9.778082294558654
[[2.09663383]
 [3.03001441]
 [3.34823183]
 [1.23278659]
 [2.78971159]
 [4.04158717]]
[[3.67596471]
 [3.10970126]
 [2.54701889]
 [2.19306089]
 [3.01453605]
 [2.54020329

In [2]:
a = tf.Variable(1)
b = tf.constant(2)
c = a + b

with tf.control_dependencies([c]):
    assign = tf.assign(a, 5)

sess = tf.Session()
for i in range(10):
    sess.run(tf.global_variables_initializer())
    print(sess.run([assign, c]))

[5, 3]
[5, 3]
[5, 3]
[5, 3]
[5, 3]
[5, 3]
[5, 3]
[5, 3]
[5, 3]
[5, 3]
